In [4]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import os 
import csv 
import math


In [5]:
#Part B SCT WALMART HACKATHON 
!pip install typing


  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26325 sha256=3f24c7740d8624e332d039223178c5a42bb679b9a1ab198d5486ffb9b6faecfd
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\fa\17\1f\332799f975d1b2d7f9b3f33bbccf65031e794717d24432caee
Successfully built typing


In [6]:
!pip install geopy

In [9]:
from typing import List
from geopy.distance import geodesic
import itertools

In [10]:
from typing import Tuple

In [11]:
def haversine_dist(point1: Tuple[float, float], point2: Tuple[float,float]) -> float:
    #calculate the haversine distance b/w 2 points
    
    lat1, lon1=point1
    lat2, lon2=point2
    R=6371 #radius of earth
    
    dlat=math.radians(lat2-lat1)
    dlon=math.radians(lon2-lon1)
    
    a=(
        math.sin(dlat/2)*math.sin(dlat/2)+math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)*math.sin(dlon/2)
    )
    c=2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return R*c

In [14]:
file_path='D:/Users/User/Downloads/sct_hackathon'
def read_input_data(file_path:str) -> List[dict]:
    #read the input data from a CSV file
    orders = []
    file_name = file_path.split('/')[-1]
    with open(file_path,mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            order = {
                'order_id':row['order_id'],
                'lng':float(row['lng']),
                'lat':float(row['lat']),
                'depot_lat':float(row['depot_lat'])
            }
            orders.append(order)
    return orders      

In [15]:
def calculate_total_distance(orders: List[dict], routes: List[List[dict]]) -> float:
    
    #Calculate the total distance traveled across all vehicles
    
    total_distance = 0
    for route in routes:
        previous_order = route[0]
        for current_order in route[1:]:
            total_distance += calculate_distance(previous_order, current_order)
            previous_order = current_order
        total_distance += calculate_distance(previous_order, route[0])
    return round(total_distance, 2)

In [19]:
import pandas as pd
import os

# Specify the folder path containing the CSV files
folder_path = r'D:/Users/User/Downloads/sct_hackathon'  # Replace 'your_folder_path' with the actual folder path

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV file
        file_path = os.path.join(folder_path, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Print or use the combined DataFrame as needed
print(combined_df.head())  # Display the first few rows of the combined DataFrame


   order_id        lng       lat  depot_lat  depot_lng
0   2556635  126.53566  43.82912    43.8502   126.5445
1   1507929  126.54484  43.84685    43.8502   126.5445
2   2234840  126.51653  43.81547    43.8502   126.5445
3    511292  126.56568  43.84987    43.8502   126.5445
4   3846410  126.55650  43.84634    43.8502   126.5445


In [21]:
def calculate_distance_matrix(orders: List[dict]) -> List[List[float]]:
    """
    Calculate the distance matrix for all orders
    """
    n = len(orders)
    distance_matrix = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i, n):
            distance = calculate_distance(orders[i], orders[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix

In [22]:
def calculate_initial_routes(orders: List[dict], capacity: int) -> List[List[dict]]:
    """
    Calculate the initial routes using the savings algorithm
    """
    n = len(orders)
    distance_matrix = calculate_distance_matrix(orders)
    savings = []
    for i in range(n):
        for j in range(i+1, n):
            savings.append((distance_matrix[i][0] + distance_matrix[0][j] - distance_matrix[i][j], i, j))
    savings.sort(reverse=True)
    routes = [[] for _ in range(2)]
    for s in savings:
        i, a, b = s[1:]
        if len(routes[0]) <= capacity and len(routes[1]) <= capacity:
            if not routes[0] or distance_matrix[routes[0][-1]['order_id']][a] > distance_matrix[routes[0][-1]['order_id']][b]:
                routes[0].append(orders[a])
                routes[1].append(orders[b])
            else:
                routes[0].append(orders[b])
                routes[1].append(orders[a])
        elif len(routes[0]) <= capacity:
            routes[0].append(orders[a])
            routes[1].append

In [23]:
def calculate_routes(orders: List[dict], capacity: int) -> List[List[dict]]:
    """
    Calculate the routes using the 2-opt algorithm
    """
    routes = calculate_initial_routes(orders, capacity)
    n = len(orders)
    improved = True
    while improved:
        improved = False
        for i in range(len(routes)):
            for j in range(1, len(routes[i])-1):
                for k in range(j+1, len(routes[i])):
                    new_routes = routes.copy()
                    new_routes[i][j:k] = reversed(new_routes[i][j:k])
                    new_distance = calculate_total_distance(new_routes, orders)
                    if new_distance < calculate_total_distance(routes, orders):
                        routes = new_routes
                        improved = True
    return routes

In [26]:
def write_output_data(file_path: str, orders: List[dict], routes: List[List[dict]]) -> None:
    """
    Write the output data to a CSV file
    """
    with open(file_path, mode='w', newline='') as file:
        fieldnames = ['vehicle_num', 'dlvr_seq_num', 'order_id', 'lng', 'lat', 'depot_lat', 'depot_lng']
        csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
        csv_writer.writeheader()
        for i, route in enumerate(routes):
            for j, order in enumerate(route):
                csv_writer.writerow({
                    'vehicle_num': i+1,
                    'dlvr_seq_num': j+1,
                    'order_id': order['order_id'],
                    'lng': order['lng'],
                    'lat':order['lat'],
                    'depot_lat':order['depot_lat'],
                    'depot_lng':order['depot_lng']
                })        
                    
                

In [27]:
def calculate_total_distance_travelled(datasets: List[Tuple[str, List[dict]]]) -> None:
    """
    Calculate the total distance travelled across all the vehicles for each input dataset
    and write the output to a CSV file
    """
    output_file_path = 'part_b_routes_distance_travelled.csv'
    with open(output_file_path, mode='w', newline='') as file:
        fieldnames = ['Dataset', 'Vehicle 1 Route', 'Distance', 'Vehicle 2 Route', 'Distance', 'Total Distance']
        csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
        csv_writer.writeheader()

        for i, (file_path, orders) in enumerate(datasets):
            routes = calculate_routes(orders, capacity=20)
            total_distance = calculate_total_distance(routes)

            csv_writer.writerow({
                'Dataset': f'part_b_input_dataset_{i+1}',
                'Vehicle 1 Route': ', '.join(order['order_id'] for order in routes[0]),
                'Distance': round(calculate_total_distance(routes[0]), 2),
                'Vehicle 2 Route': ', '.join(order['order_id'] for order in routes[1]),
                'Distance': round(calculate_total_distance(routes[1]), 2),
                'Total Distance': total_distance
            })